# Imports

In [10]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [11]:
clicks = pd.read_pickle("../../../../data/tp2/clicks_tp2_formateado.pkl")

---

# Armado de ventanas

In [12]:
for ventana_nro in range(1,6):
    clicks["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= clicks["created"].dt.day) & ((clicks["created"].dt.day) <= (19+ventana_nro)))

---

# Creación de features

## Creación de sets de entrenamiento

In [13]:
Xs = {}
for ventana_nro in range(1,6):
    Xs[ventana_nro] = clicks.loc[clicks["ventana_{}".format(ventana_nro)]]["ref_hash"].drop_duplicates().to_frame().set_index("ref_hash")

---

## Función generadora de features

In [14]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos):
    for ventana_nro in range(1, 6):
        feature = generador_feature(dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]], ventana_nro)
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True)

---

## Cantidad de clicks discriminando por períodos

In [15]:
# Creo columnas para luego separar
clicks['hora'] = clicks['created'].dt.hour
clicks['dia'] = clicks['created'].dt.day
clicks['dia_de_la_semana'] = clicks['created'].dt.dayofweek

In [16]:
def cantidad_apariciones_por_dia(dataframe, nro_ventana):
    dias = dataframe["dia"].drop_duplicates
    apariciones_por_dia = dataframe.groupby(["ref_hash", "dia"]).agg(({"created" : "count"}))
    apariciones_por_dia = apariciones_por_dia.unstack().fillna(0).astype(np.int64)
    apariciones_por_dia.columns = ["clicks_en_dia_{}".format(dia) for dia in range(1,4)]
    return apariciones_por_dia

In [17]:
generar_feature_en_ventanas(clicks, cantidad_apariciones_por_dia, Xs)

---

---

# Guardado de los sets de entrenamiento

In [18]:
for ventana_nro in range(1, 6):
    Xs[ventana_nro].to_pickle("../../features/clicks_v_{}.pkl".format(ventana_nro))
    Xs[ventana_nro].to_csv("../../features/clicks_v_{}.csv".format(ventana_nro))

---